# Regressão Logística

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('log').getOrCreate()

24/01/03 16:53:23 WARN Utils: Your hostname, Valhalla-Zorin resolves to a loopback address: 127.0.1.1; using 192.168.100.36 instead (on interface enp4s0)
24/01/03 16:53:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 16:53:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.read.format('libsvm').load('Data/sample_libsvm_data.txt')

24/01/03 16:56:12 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
log_reg_model = LogisticRegression()

In [7]:
fitted_model = log_reg_model.fit(data)

24/01/03 16:57:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/03 16:57:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [8]:
summary = fitted_model.summary

In [10]:
summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

### Treinando o modelo com randomsplit dos dados

In [12]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [13]:
log_reg_model_2 = LogisticRegression(labelCol='label',featuresCol='features')

In [14]:
fitted_model_2 = log_reg_model_2.fit(train_data)

In [15]:
predictions = fitted_model_2.evaluate(test_data)

In [18]:
predictions.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[124,125,126...|[31.4040459847825...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[30.9095257307243...|[0.99999999999996...|       0.0|
|  0.0|(692,[124,125,126...|[24.4301566358813...|[0.99999999997544...|       0.0|
|  0.0|(692,[124,125,126...|[22.3709238091452...|[0.99999999980749...|       0.0|
|  0.0|(692,[125,126,127...|[28.3670496073496...|[0.99999999999952...|       0.0|
|  0.0|(692,[126,127,128...|[23.0736458626295...|[0.99999999990466...|       0.0|
|  0.0|(692,[126,127,128...|[29.9854014557574...|[0.99999999999990...|       0.0|
|  0.0|(692,[151,152,153...|[32.1675681869793...|[0.99999999999998...|       0.0|
|  1.0|(692,[99,100,101,...|[-5.9339475463147...|[0.00264101476575...|       1.0|
|  1.0|(692,[119

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [20]:
bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')

In [21]:
results = bin_evaluator.evaluate(predictions.predictions)

BinaryEvaluator retorna um valor da área abaixo da curva ROC, sendo 1.0 um resultado perfeito

In [23]:
results

1.0